In [1]:
import os
import shutil
import supervision as sv

root_path = "D:/Downloads/street-facilities-selected"

# 데이터셋 로드
ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{root_path}/transformed-images",
    annotations_directory_path=f"{root_path}/transformed-labels-txt",
    data_yaml_path=f"{root_path}/street-facilities.yaml"
)

# 데이터셋 분리 (train 70%, val 15 % test 15%)
train_ds, val_test_ds = ds.split(split_ratio=0.7, random_state=1014, shuffle=True)
val_ds, test_ds = val_test_ds.split(split_ratio=0.5, random_state=535, shuffle=True)

print("train_ds length : ", len(train_ds))
print("val_ds length : ", len(val_ds))
print("test_ds length : ", len(test_ds))

train_ds length :  6844
val_ds length :  1467
test_ds length :  1467


### 분할된 데이터셋 클래스 분포 확인

In [2]:
import numpy as np

def count_classes(dataset : sv.DetectionDataset):
    n_classes = len(dataset.classes)
    
    count = np.zeros(n_classes, dtype=np.int16)
    for _, _, annotation in dataset.__iter__():
        # 사진 하나에 존재하는 0개 이상의 class object 들을 각각 센다
        # print(annotation) 
        for label in annotation.class_id:
            count[label] += 1

    return count

# names:
#   0: braille-block-defect
#   1: sidewalk-block-defect
#   2: bicycle-road-defect

print("class 0 (점자블록 파손부), 1(보도블럭 파손부), 2(자전거도로 파손부)")
for ds in [train_ds, val_ds, test_ds]:
    print(count_classes(ds))

class 0 (점자블록 파손부), 1(보도블럭 파손부), 2(자전거도로 파손부)


KeyboardInterrupt: 

저장

In [ ]:
# 저장할 디렉토리 경로 설정
root_path = "D:/Downloads/street-facilities-5"
train_images_dir = f"{root_path}/images/train"
train_labels_dir = f"{root_path}/labels/train"

val_images_dir = f"{root_path}/images/val"
val_labels_dir = f"{root_path}/labels/val"

test_images_dir = f"{root_path}/images/test"
test_labels_dir = f"{root_path}/labels/test"

# 디렉토리 생성 (존재하지 않는 경우에만)
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)

os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# DetectionDataset 객체의 이미지 및 라벨을 복사하여 저장
def save_dataset(dataset, images_dir_to_save, labels_dir_to_save):
    i = 1
    total = len(dataset)
    for image_path, image, annotation in dataset.__iter__():
        if i % 200 == 1 : print(f"{i} / {total}")
        # image_path : path/image_filename.jpg or .jpeg
        image_filename = os.path.basename(image_path) # 이미지 파일 이름
        label_filename = os.path.splitext(image_filename)[0] + ".txt" # 확장자명만 변경 (.jpg or .jpeg -> .txt)

        # label_path 조합
        label_path = "\\".join(image_path.split("\\")[:-2] + ["transformed-labels-txt", label_filename])
        
        # 이미지 및 라벨을 각각 지정된 폴더로 복사
        shutil.copy(image_path, os.path.join(images_dir_to_save, image_filename))
        shutil.copy(label_path, os.path.join(labels_dir_to_save, label_filename))
        i += 1


# train 데이터셋 저장
save_dataset(train_ds, train_images_dir, train_labels_dir)
# validation 데이터셋 저장
save_dataset(val_ds, val_images_dir, val_labels_dir)
# test 데이터셋 저장
save_dataset(test_ds, test_images_dir, test_labels_dir)

print("Train/Validation/Test 데이터셋 저장 완료.")

1 / 6844
201 / 6844
401 / 6844
601 / 6844
801 / 6844
1001 / 6844
1201 / 6844
1401 / 6844
1601 / 6844
1801 / 6844
2001 / 6844
2201 / 6844
2401 / 6844
2601 / 6844
2801 / 6844
3001 / 6844
3201 / 6844
3401 / 6844
3601 / 6844
3801 / 6844
4001 / 6844
4201 / 6844
4401 / 6844
4601 / 6844
4801 / 6844
5001 / 6844
5201 / 6844
5401 / 6844
5601 / 6844
5801 / 6844
6001 / 6844
6201 / 6844
6401 / 6844
6601 / 6844
6801 / 6844
1 / 1467
201 / 1467
401 / 1467
601 / 1467
801 / 1467
1001 / 1467
1201 / 1467
1401 / 1467
1 / 1467
201 / 1467
401 / 1467
601 / 1467
801 / 1467
1001 / 1467
1201 / 1467
1401 / 1467
Train/Validation/Test 데이터셋 저장 완료.


저장 완료 후 데이터셋 분포 빠르게 확인

In [5]:
import os 
import numpy as np

root_path = "D:/Downloads/street-facilities-5-db/labels"

sub_path = ["train", "val", "test"]

class_mapping = {
    9 : 0,
    12: 1, 
    13 : 2,
}

cnt_classes = np.zeros((3,3), dtype=int)
negative_samples = np.zeros((3,3), dtype=int)

for k, sub in enumerate(sub_path):
    label_dir = os.path.join(root_path, sub)

    labels = [label for label in os.listdir(label_dir) if label.endswith('.txt')]
    total_cnt = len(labels)

    for i, label in enumerate(labels):
        if i % 300 == 0 :
            print(f"{i+1} / {total_cnt}")

        ctype = int(label.split("_")[1])

        # if (int(label.split("_")[2]) == 0): # 정상(0) 라벨
        #     ctype = int(label.split("_")[1])
        #     negative_samples[k][class_mapping[ctype]] += 1

        label_path = os.path.join(label_dir, label)
        # 정상 라벨 데이터
        if (os.path.isfile(label_path) and os.path.getsize(label_path) == 0):
            negative_samples[k][class_mapping[ctype]] += 1
            continue

        # 불량 라벨 데이터
        with open(label_path, "r+", encoding="utf-8") as f:
            # 기존 내용 읽기 및 수정        
            for line in f:
                elems = line.split(" ")
                # 클래스 분포 집계 (0, 1, 2)
                # cnt_classes[k][class_mapping(ctype)] += 1
                cnt_classes[k][int(float(elems[0]))] += 1

print()
for i, type in enumerate(sub_path):
    print(type)
    print(cnt_classes[i])
    print("negative sample")
    print(negative_samples[i])

1 / 13689
301 / 13689
601 / 13689
901 / 13689
1201 / 13689
1501 / 13689
1801 / 13689
2101 / 13689
2401 / 13689
2701 / 13689
3001 / 13689
3301 / 13689
3601 / 13689
3901 / 13689
4201 / 13689
4501 / 13689
4801 / 13689
5101 / 13689
5401 / 13689
5701 / 13689
6001 / 13689
6301 / 13689
6601 / 13689
6901 / 13689
7201 / 13689
7501 / 13689
7801 / 13689
8101 / 13689
8401 / 13689
8701 / 13689
9001 / 13689
9301 / 13689
9601 / 13689
9901 / 13689
10201 / 13689
10501 / 13689
10801 / 13689
11101 / 13689
11401 / 13689
11701 / 13689
12001 / 13689
12301 / 13689
12601 / 13689
12901 / 13689
13201 / 13689
13501 / 13689
1 / 2933
301 / 2933
601 / 2933
901 / 2933
1201 / 2933
1501 / 2933
1801 / 2933
2101 / 2933
2401 / 2933
2701 / 2933
1 / 2934
301 / 2934
601 / 2934
901 / 2934
1201 / 2934
1501 / 2934
1801 / 2934
2101 / 2934
2401 / 2934
2701 / 2934

train
[ 7667 10625  7542]
negative sample
[ 507 1525 1272]
val
[1506 2451 1513]
negative sample
[119 342 277]
test
[1619 2268 1739]
negative sample
[102 299 281]
